In [1]:
# Original dataset taken from: https://www.kaggle.com/mkechinov/ecommerce-events-history-in-cosmetics-shop?select=2019-Dec.csv
# Original dataset attributable to: https://rees46.com/

# Notebook used by Clayton Cohn to clean data before use
# Data will be used for item-item collaborative filtering recommender system

import pandas as pd

DATA_PATH = "/Users/claytoncohn/Dropbox/New/DePaul/DSC478/FinalProject/"

In [2]:
# First import raw data

import numpy as np

df = pd.read_csv("original_data.csv", header=0, names=["event_time", "event_type", "product_id", 
                                              "category_id", "category_code", "brand", "price", "user_id", "user_session"])
print(df.shape)
df.head()

(3533286, 9)


,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-12-01 00:00:00 UTC,remove_from_cart,5712790,1487580005268456287,NaN,f.o.x,6.27,576802932,51d85cb0-897f-48d2-918b-ad63965c12dc
1,2019-12-01 00:00:00 UTC,view,5764655,1487580005411062629,NaN,cnd,29.05,412120092,8adff31e-2051-4894-9758-224bfa8aec18
2,2019-12-01 00:00:02 UTC,cart,4958,1487580009471148064,NaN,runail,1.19,494077766,c99a50e8-2fac-4c4d-89ec-41c05f114554
3,2019-12-01 00:00:05 UTC,view,5848413,1487580007675986893,NaN,freedecor,0.79,348405118,722ffea5-73c0-4924-8e8f-371ff8031af4
4,2019-12-01 00:00:07 UTC,view,5824148,1487580005511725929,NaN,NaN,5.56,576005683,28172809-7e4a-45ce-bab0-5efa90117cd5


In [3]:
# Drop unwanted columns
# For item-item system, I am only concerned with who bought what

df = df.drop(columns=['event_time','category_code', 'user_session', "category_id", "brand", "price"])
print(df.shape)
df.head(10)

(3533286, 3)


,event_type,product_id,user_id
0,remove_from_cart,5712790,576802932
1,view,5764655,412120092
2,cart,4958,494077766
3,view,5848413,348405118
4,view,5824148,576005683
5,view,5773361,560109803
6,cart,5629988,579966747
7,view,5807805,576005683
8,view,5588608,546170008
9,cart,5335,494077766


In [4]:
# Get rid of rows with NaN values (there aren't any in this dataset)

df.dropna(how='any', inplace=True)
print(df.shape)
df.head(10)

(3533286, 3)


,event_type,product_id,user_id
0,remove_from_cart,5712790,576802932
1,view,5764655,412120092
2,cart,4958,494077766
3,view,5848413,348405118
4,view,5824148,576005683
5,view,5773361,560109803
6,cart,5629988,579966747
7,view,5807805,576005683
8,view,5588608,546170008
9,cart,5335,494077766


In [5]:
# Only concerned with purchases, can ignore view, cart, remove from cart.

df = df.loc[df['event_type'] == "purchase"]
print(df.shape)
df.head(10)

(213176, 3)


,event_type,product_id,user_id
557,purchase,4554,473613801
558,purchase,4600,473613801
559,purchase,4607,473613801
560,purchase,4640,473613801
561,purchase,4689,473613801
562,purchase,4768,473613801
563,purchase,4820,473613801
564,purchase,4870,473613801
565,purchase,5762834,473613801
566,purchase,5811642,473613801


In [6]:
# Drop event_type after purchases are filtered out

df = df.drop(columns=["event_type"])
print(df.shape)
df.head(10)

(213176, 2)


,product_id,user_id
557,4554,473613801
558,4600,473613801
559,4607,473613801
560,4640,473613801
561,4689,473613801
562,4768,473613801
563,4820,473613801
564,4870,473613801
565,5762834,473613801
566,5811642,473613801


In [7]:
# Factorize user labels for readability and ease of computation

user_labels, _ = pd.factorize(df["user_id"])
user_labels[:100]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4,
       4, 4, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6])

In [8]:
df["user_id"] = user_labels
df.head(10)

,product_id,user_id
557,4554,0
558,4600,0
559,4607,0
560,4640,0
561,4689,0
562,4768,0
563,4820,0
564,4870,0
565,5762834,0
566,5811642,0


In [9]:
# Factorize product labels for readability and ease of computation

product_labels, _ = pd.factorize(df["product_id"])
product_labels[:100]

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
       85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])

In [10]:
df["product_id"] = product_labels
df.head(10)

,product_id,user_id
557,0,0
558,1,0
559,2,0
560,3,0
561,4,0
562,5,0
563,6,0
564,7,0
565,8,0
566,9,0


In [11]:
print(df["product_id"].max())

26814


In [12]:
# Need to cut a lot of products due to memory constraints
# Product matrix pretty sparsely populated after 1000
# Select top 1000 most-purchased products

products = df['product_id'].value_counts()[:1000].sort_values(ascending=False)
print(products.shape)
products

(1000,)


352      1659
104       786
364       714
126       621
388       620
         ... 
3529       35
610        35
3235       35
7714       35
13678      35
Name: product_id, Length: 1000, dtype: int64

In [13]:
# Get indexes of 1000 most-purchased products

products_arr = np.array(products.index)
products_arr[:100]

array([  352,   104,   364,   126,   388,   146,   509,   412,   985,
         932,   386,   413,   387,  2654,    72,   729, 12090,    71,
       12208,   493,   343,   468,  1437,  1098, 10746,     1,   307,
        1431,   534,   169,  2657,   837,    55,   313,   314,   640,
        2777,  1445,  1763,  2799,   590,   718,     5,   698,   500,
         402,   967,   502,   762,  2569,  1056,   139, 12209,  1105,
        1787,  1125,  2367,    42,  1855,  1635,     2,  1180,   585,
         445,   315,   561,  1853,   728, 14240,   630,   618,  1797,
         530,   882,   920, 12569,  2567,   800,    68,   129,  1134,
         363,  1191,  5376,  1634,  1170,   300,  2986,   130,   629,
         101,   656,   197,  3363,   943,   657,   869,   918,   423,
       18249])

In [14]:
# Keep only those products whose indexes are in the above array

df = df[df['product_id'].isin(products_arr)]
print(df.shape)
df.head(10)

(79819, 2)


,product_id,user_id
557,0,0
558,1,0
559,2,0
562,5,0
563,6,0
568,11,0
571,14,0
572,15,0
573,16,0
574,17,0


In [15]:
print("Product Count:",df["product_id"].nunique())

Product Count: 1000


In [16]:
# Refactorize user labels

user_labels, _ = pd.factorize(df["user_id"])
user_labels[:100]

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  1,  2,
        3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  4,
        4,  5,  6,  6,  7,  7,  7,  7,  8,  8,  8,  8,  8,  8,  8,  9,  9,
        9,  9, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 11, 11, 11, 11,
       11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 12, 12, 12, 12, 12, 12, 12,
       12, 13, 13, 13, 14, 14, 14, 14, 14, 14, 14, 15, 15, 15, 15])

In [17]:
df["user_id"] = user_labels
df.head(10)

,product_id,user_id
557,0,0
558,1,0
559,2,0
562,5,0
563,6,0
568,11,0
571,14,0
572,15,0
573,16,0
574,17,0


In [18]:
# Refactorize product labels

product_labels, _ = pd.factorize(df["product_id"])
product_labels[:100]

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 34, 48, 49,
       50, 51, 52, 53, 54, 55, 56, 25, 57, 58, 59, 60, 61, 62, 63, 21, 64,
       65, 24, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 20, 77, 78, 79,
       80, 81, 59, 60, 27, 28, 82, 59, 60, 83, 73, 84, 18, 85, 86])

In [19]:
df["product_id"] = product_labels
df.head(10)

,product_id,user_id
557,0,0
558,1,0
559,2,0
562,3,0
563,4,0
568,5,0
571,6,0
572,7,0
573,8,0
574,9,0


In [20]:
# Used for verification

print("Users:",df["user_id"].nunique())
print("Products:",df["product_id"].nunique())
print("Shape:",df.shape)

Users: 19209
Products: 1000
Shape: (79819, 2)


In [21]:
# Save final matrix to csv file

df.to_csv('cosmetics_data.csv', index=False)

In [22]:
# Read back csv to make sure there are no problems

df = pd.read_csv("cosmetics_data.csv", header=0)
print(df.shape)
df.head(10)

(79819, 2)


,product_id,user_id
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
5,5,0
6,6,0
7,7,0
8,8,0
9,9,0


In [23]:
# Verify numbers are same as above

print("Users:",df["user_id"].nunique())
print("Products:",df["product_id"].nunique())
print("Shape:",df.shape)

Users: 19209
Products: 1000
Shape: (79819, 2)
